In [43]:
import pandas as pd
import numpy as np
import h5py
from pandas import concat
from datetime import datetime
from sklearn.svm import SVR
from keras.layers import Input, Dense, LSTM
from keras import regularizers
from keras.models import Model, load_model, Sequential
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.linear_model import LogisticRegression #logistic regression
import matplotlib.pyplot as plt
import math
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [21]:
# load data
def parse(x):
    return datetime.strptime(x, '%Y %m %d %H')
dataset = pd.read_csv('merged_final_v2.csv',  parse_dates = [['year', 'month', 'day', 'hour']], index_col=0, date_parser=parse)
dataset.drop('Unnamed: 0', axis=1, inplace=True)
# manually specify column names
dataset.index.name = 'date_1'
# drop the first 24 hours
dataset = dataset[24:]
new_dataset = dataset[['pm2.5','dew_point','air_temp','air_pressure','wind_dir','wind_speed','cumulative_snow_hours','cumulative_rain_hours']]
#drop column wind_dir
new_dataset = new_dataset.drop('wind_dir', 1)
# summarize first 5 rows
print(new_dataset.head(5))


/Applications/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                     pm2.5  dew_point  air_temp  air_pressure  wind_speed  \
date_1                                                                      
2010-01-02 00:00:00  129.0        -16      -4.0        1020.0        1.79   
2010-01-02 01:00:00  148.0        -15      -4.0        1020.0        2.68   
2010-01-02 02:00:00  159.0        -11      -5.0        1021.0        3.57   
2010-01-02 03:00:00  181.0         -7      -5.0        1022.0        5.36   
2010-01-02 04:00:00  138.0         -7      -5.0        1022.0        6.25   

                     cumulative_snow_hours  cumulative_rain_hours  
date_1                                                             
2010-01-02 00:00:00                    0.0                    0.0  
2010-01-02 01:00:00                    0.0                    0.0  
2010-01-02 02:00:00                    0.0                    0.0  
2010-01-02 03:00:00                    1.0                    0.0  
2010-01-02 04:00:00                    2.0          

In [28]:
new_dataset.isnull().sum()

pm2.5                    0
dew_point                0
air_temp                 0
air_pressure             0
wind_speed               0
cumulative_snow_hours    0
cumulative_rain_hours    0
dtype: int64

In [40]:
 def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	agg = concat(cols, axis=1)
	agg.columns = names
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 
values = new_dataset.values


values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 1, 1)
print(reframed.head())
values = reframed.values

n_train_hours = 1825 * 24

train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
train_X, train_y = train[:, 1:], train[:, 1:]
test_X, test_y = test[:, :-1], test[:, -1]
train_X = train_X.reshape((train_X.shape[0], train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
 


   var1(t-1)  var2(t-1)  var3(t-1)  var4(t-1)  var5(t-1)  var6(t-1)  \
1   0.129779   0.031431   0.015356   0.003061   0.000179   0.000000   
2   0.148893   0.031528   0.015356   0.003061   0.000268   0.000000   
3   0.159960   0.031915   0.015257   0.003167   0.000357   0.000000   
4   0.182093   0.032302   0.015257   0.003272   0.000536   0.010101   
5   0.138833   0.032302   0.015257   0.003272   0.000625   0.020202   

   var7(t-1)   var1(t)   var2(t)   var3(t)   var4(t)   var5(t)   var6(t)  \
1        0.0  0.148893  0.031528  0.015356  0.003061  0.000268  0.000000   
2        0.0  0.159960  0.031915  0.015257  0.003167  0.000357  0.000000   
3        0.0  0.182093  0.032302  0.015257  0.003272  0.000536  0.010101   
4        0.0  0.138833  0.032302  0.015257  0.003272  0.000625  0.020202   
5        0.0  0.109658  0.032302  0.015159  0.003272  0.000714  0.030303   

   var7(t)  
1      0.0  
2      0.0  
3      0.0  
4      0.0  
5      0.0  
(43800, 13) (43800,) (25145, 13) (2514

In [ ]:
# y = df.temp.values
# X = df.iloc[:, 1:].values

# # Train on 70% of the data
# train_idx = int(len(df) * .7)

# # create train and test data
# X_train, y_train, X_test, y_test = X[:train_idx], y[:train_idx], X[train_idx:], y[:train_idx]

# # fit and predict
# clf = SVR()
# clf.fit(X_train, y_train)

# clf.predict(X_test)


In [44]:
clf = SVR()
clf.fit(train_X, train_y)

y_pred = clf.predict(test_X)

rmse = sqrt(mean_squared_error(y_pred, test_y))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.114


In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()
 
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [51]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 
# load dataset
values = new_dataset.values
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# specify the number of lag hours
n_hours = 3
n_features = 7
# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 1)
print(reframed.shape)
 
# split into train and test sets
values = reframed.values

print(reframed)

(68943, 28)
       var1(t-3)  var2(t-3)  var3(t-3)  var4(t-3)  var5(t-3)  var6(t-3)  \
3       0.129779   0.031431   0.015356   0.003061   0.000179   0.000000   
4       0.148893   0.031528   0.015356   0.003061   0.000268   0.000000   
5       0.159960   0.031915   0.015257   0.003167   0.000357   0.000000   
6       0.182093   0.032302   0.015257   0.003272   0.000536   0.010101   
7       0.138833   0.032302   0.015257   0.003272   0.000625   0.020202   
8       0.109658   0.032302   0.015159   0.003272   0.000714   0.030303   
9       0.105634   0.032302   0.015159   0.003378   0.000893   0.040404   
10      0.124748   0.032302   0.015257   0.003483   0.001072   0.000000   
11      0.120724   0.032205   0.015159   0.003483   0.001251   0.000000   
12      0.132797   0.032302   0.015257   0.003589   0.001430   0.000000   
13      0.140845   0.032302   0.015257   0.003694   0.001743   0.010101   
14      0.152918   0.032205   0.015257   0.003694   0.002056   0.000000   
15      0.148

In [61]:
n_train_hours = 1825 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)
# reshape input to be 3D [samples, timesteps, features]
# train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
# test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
 

(43800, 21) 43800 (43800,)
(43800, 21) (43800,) (25143, 21) (25143,)


In [62]:
clf = SVR()
clf.fit(train_X, train_y)

y_pred = clf.predict(test_X)

rmse = sqrt(mean_squared_error(test_y, y_pred))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.085


In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()
 
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, -7:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -7:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)